In [4]:
import json
from openai import OpenAI
import requests

In [2]:
with open('conf.json') as conf_json:
    conf = json.load(conf_json)

In [30]:
client = OpenAI(
  api_key=conf['key_openai']
)

In [ ]:
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  store=True,
  messages=[
    {"role": "user", 
     "content": "write a haiku about ai"}
  ]
)

print(completion.choices[0].message);


ChatCompletionMessage(content='Silent circuits hum,  \nWisdom born from code and light,  \nDreams in silicon.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [93]:
def get_employees(url = conf['url']):
    response = requests.get(url).json()
    return response

def post_employee(url = conf['url'], employee_data={}): 
    response = requests.post(url, json=employee_data).json()
    return response

In [92]:
post_employee(employee_data={'name':'Jose'})

{'name': 'Jose'}


{'id': 4, 'name': 'Jose'}

In [76]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_employees",
            "description": "Get the dictionary of employees names",
        },   
    },
    {
        "type": "function",
        "function": {
            "name": "post_employee",
            "description": "Add a new employee name to the list of employees",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Name of the employee"
                    }
                },
                "required": ["name"],
            },
        },   
    }
]

In [149]:
def run_function(completion):
    if completion.choices[0].finish_reason == 'tool_calls':
        function = completion.choices[0].message.tool_calls[0].function.name
        args = completion.choices[0].message.tool_calls[0].function.arguments
        if args == '{}':
            return eval(function)()
        else:
            print(eval(function)(employee_data = json.loads(args)))
    else:
        print('não necessário executar função')

In [148]:
messages = [{"role": "user", "content": "How many employees do we have?"}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [150]:
completion

ChatCompletion(id='chatcmpl-AwGviji9fOX0pAwHjSrFoTM5cqFWB', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_EdDr3PHuDsH5BHr8656ZBvDE', function=Function(arguments='{}', name='get_employees'), type='function')]))], created=1738451466, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_50cad350e4', usage=CompletionUsage(completion_tokens=12, prompt_tokens=77, total_tokens=89, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [159]:
messages.append(completion.choices[0].message)

In [152]:
completion.choices[0].message.tool_calls[0]

ChatCompletionMessageToolCall(id='call_EdDr3PHuDsH5BHr8656ZBvDE', function=Function(arguments='{}', name='get_employees'), type='function')

In [153]:
result = run_function(completion)

In [154]:
result

[{'id': 1, 'name': 'Ashley'},
 {'id': 2, 'name': 'Kate'},
 {'id': 3, 'name': 'Joe'},
 {'id': 4, 'name': 'Marco'}]

In [160]:
messages.append({
        "role": "tool",
        "tool_call_id": completion.choices[0].message.tool_calls[0].id,
        "content": str(result)
    })

In [161]:
messages

[{'role': 'user', 'content': 'How many employees do we have?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_EdDr3PHuDsH5BHr8656ZBvDE', function=Function(arguments='{}', name='get_employees'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_EdDr3PHuDsH5BHr8656ZBvDE',
  'content': "[{'id': 1, 'name': 'Ashley'}, {'id': 2, 'name': 'Kate'}, {'id': 3, 'name': 'Joe'}, {'id': 4, 'name': 'Marco'}]"}]

In [162]:
completion_qtd = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [164]:
completion_qtd.choices[0].message.content

'We currently have 4 employees.'

##### Checando para pergunta sem relação com as tools como seria o retorno

In [27]:
messages = [{"role": "user", "content": "What is AI?"}]

completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [29]:
completion_2.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think, learn, and problem-solve like humans. These systems utilize algorithms and computational models to perform tasks that typically require human intelligence, such as understanding natural language, recognizing patterns, learning from experience, and making decisions. AI applications can be found in various fields, including robotics, natural language processing, computer vision, and more, aiming to automate tasks, enhance efficiency, and provide insights that would be challenging for humans to discover unaided.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))

In [58]:
run_function(completion_2)

não necessário executar função


#### Testando função com método POST

In [140]:
messages = [{"role": "user", "content": "Please, can you add a new user named Marco?"}]

In [127]:
messages = [{"role": "user", "content": "Please, can you add a new user named Marco?"}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [78]:
completion.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_mqz48H5HSls4TeOO5RfZGccD', function=Function(arguments='{"name":"Marco"}', name='post_employee'), type='function')]))

In [103]:
result = run_function(completion)

{'id': 4, 'name': 'Marco'}


In [104]:
result

#### Retorno da mensagem

In [128]:
completion.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_fmUbuNHXgQmaJ1uHzserr1UY', function=Function(arguments='{"name":"Marco"}', name='post_employee'), type='function')]

In [141]:
messages.append(completion.choices[0].message)
messages.append({
        "role": "tool",
        "tool_call_id": completion.choices[0].message.tool_calls[0].id,
        "content": '201'
    })


In [142]:
messages

[{'role': 'user', 'content': 'Please, can you add a new user named Marco?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_fmUbuNHXgQmaJ1uHzserr1UY', function=Function(arguments='{"name":"Marco"}', name='post_employee'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_fmUbuNHXgQmaJ1uHzserr1UY',
  'content': '201'}]

In [143]:
completion_final = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [146]:
completion_final.choices[0].message.content

'The new employee named Marco has been successfully added to the list.'